In [218]:
import pandas as pd
import re
import urllib.request
from scrapy.selector import Selector
from pymorphy2 import MorphAnalyzer

Парсим страничку яндекса с курсом биткоина

In [219]:
html=urllib.request.urlopen('https://news.yandex.ru/quotes/60000.html?nar=').read()

In [220]:
btc_all=re.findall(r'(\d+.\d+.\d+)', Selector(text=html).xpath('/html[@class="i-ua_js_no i-ua_css_standard"]\
                          /body[@class="b-page b-page_touch_no b-page_name_quote b-page_type_quotes b-page_ajax b-page_lang_ru b-page_view_normal b-page_mobile_yes loader loader_theme_yellow-38 i-ua daas b-page__body i-global i-ua i-ua i-ua_interaction_yes loader loader_theme_yellow-38 i-ua daas i-bem"]\
                          /div[@class="b-page__inner"]\
                          /div[@class="page-content"]\
                          /div[@class="page-content__layout"]\
                          /div[@class="page-content__left"]\
                          /div[@class="quote i-bem"]\
                          /div[@class="quote__graph"]\
                          /div[@class="quote__data"]\
                          /table\
                          ').extract()[0])

In [221]:
df=pd.DataFrame.from_records(list(zip(*[iter(btc_all)]*3)), columns=['date', 'value','change'])

In [222]:
df['value']=df.value.apply(lambda x: round(float(x.replace(',','.'))))
df['change']=df.change.apply(lambda x: round(float(x.replace(',','.'))))

Получаем такую табличку

In [258]:
df.head()

,date,value,change
0,26.12.17,15200,1268
1,25.12.17,13932,162
2,24.12.17,13770,704
3,23.12.17,14474,673
4,22.12.17,13801,1725


Делаем функцию, которая считает курс доллара к биткоин, по сравнению с сегодя и вчера.
<br/>PS. т.к. слово **биткоин** pymorhy согласовывать с числом не умеет, то подсовываем для склонения слово **слон**

In [300]:
def btc_course(x,date):
    m=MorphAnalyzer()
    baks=m.parse('доллар')[0]
    verb=m.parse('составлять')[0]
    btc=m.parse('слона')[0]
    btc=btc.make_agree_with_number(int(x)).word
    
    if date=='сегодня':
        d=0
        v=verb.inflect({'sing','3per'}).word
        day='вчера'
        if df['value'][0]>df['value'][1]:
            change='больше'
        else:
            change='меньше'
    elif date=='вчера':
        d=1
        v=verb.inflect({'femn','past'}).word
        day='сегодня'
        if df['value'][1]>df['value'][0]:
            change='больше'
        else:
            change='меньше'
            
    print("По состоянию на {} стоимость {} {} {} {} {}, что на {} {} {} чем {}".format(\
                   df['date'][d],x,btc.replace('слон','биткоин'),v,str(df['value'][d]*int(x))\
                  ,baks.make_agree_with_number(int(str(df['value'][d])[-1])).word\
                  ,str(df['change'][0]*int(x)),baks.make_agree_with_number(int(str(df['change'][0])[-1])).word\
                  ,change,day))

In [301]:
btc_course('1','сегодня')

По состоянию на 26.12.17 стоимость 1 биткоина составляет 15200 долларов, что на 1268 долларов больше чем вчера


In [302]:
btc_course('2','вчера')

По состоянию на 25.12.17 стоимость 2 биткоинов составляла 27864 доллара, что на 2536 долларов меньше чем сегодня
